## Categoricals and the string cache
By the end of this lecture you will be able to:
- coordinate categorical mappings across `DataFrames` with the string cache
- filter a categorical column

We introduce the string cache here. In Section 6 on Joins and Concats we will see that the string cache is essential when combining `DataFrames` with categorical columns. We will also see that using categoricals can lead to much faster join operations than with strings.

In [1]:
import polars as pl

We create a `DataFrame` and add a categorical column called `cats`

In [2]:
df = (
    pl.DataFrame(
        {
            "strings": ["c","b","a","c"], 
            "values": [1, 2, 3, 4]
        }
    )
    .with_columns(
        pl.col("strings").cast(pl.Categorical).alias("cats")
    )
)
df

strings,values,cats
str,i64,cat
"""c""",1,"""c"""
"""b""",2,"""b"""
"""a""",3,"""a"""
"""c""",4,"""c"""


## Filtering a categorical column
We filter a categorical column for equality in the same way as for a string column

In [3]:
(
    df
    .filter(
        cats = "b"
    )
)

strings,values,cats
str,i64,cat
"""b""",2,"""b"""


If we try to filter a categorical column with `is_in` we get an `Exception`

In [ ]:
(
    df
    .filter(
        pl.col("cats").is_in(["b"])
    )
)

### Why do we get an `Exception` with `is_in`?

When we use `is_in` Polars: 
1. converts the list `["b"]` to a one-element `Series` with string dtype internally
2. casts this `Series` to `pl.Categorical`
3. uses its internal algorithms for comparing categoricals

The problem is in step 2. The mapping to categoricals for the `Series` `["b"]` is not guaranteed to be the same mapping as for `df`

## Combining categoricals with the `StringCache`
To ensure that different objects - in this case `df["cats"]` and `["b"]` have the same categorical mapping we use the `StringCache`.

The `StringCache` object:
- stores the categorical mapping
- ensures that all categorical columns use the same mapping. 

We can use the `StringCache` inside a context manager and by enabling it globally. 

We never work with the data in the `StringCache` directly.

### Using the `StringCache` inside a context-manager

A context-manager is a way to ensure certain actions happen in Python.

Everything inside the code block beginning with `with` is in the same context.

In this case
```python
with pl.StringCache():
```
ensures that everything that happens in the following code block uses the same categorical mappings. In this example it ensures that the list in `is_in` is cast to the same categorical mappings as the `strings` column of the `DataFrame`

In [ ]:
with pl.StringCache():
    df = (
        pl.DataFrame(
                {"strings": ["c","b","a","c"], "values": [1, 2, 3, 4]}
        )
        .with_columns(
            pl.col("strings").cast(pl.Categorical).alias("cats")
        )
        .filter(
            pl.col("cats").is_in(["b"])
        )
)
df

At the end of the `with` block the `StringCache` is deleted.

### Enabling the `StringCache`
We can also enable the `StringCache` to be on through a session - be aware that this can have affects beyond this script/notebook. In fact I've commented it out here because when I run my test suite with `pytest` this command changes the outputs in other notebooks!

In [ ]:
# pl.enable_string_cache()

When we use `pl.enable_string_cache()` Polars enables a `StringCache` that is used by all categorical columns until:
- the end of the session or
- you call `pl.disable_string_cache()`

You can see whether a string cache is enabled with 

In [ ]:
pl.using_string_cache()

### Context-manager or enable the string cache?
Enabling the string cache is easier than using pl.StringCache in a context-manager.

However, I recommend using the context-manager approach as:
- it makes the use of the string cache explicit in the code
- it avoids errors that can arise from setting global values

### Use cases for `pl.StringCache`

We need the string cache whenever different objects with a categorical dtype are involved. For example when:
- joining `DataFrames` with categorical dtypes
- concatenating `DataFrames` with categorical dtypes
- creating a `DataFrame` with categorical dtype from multiple files

We will see examples of these in later Sections of the course.

## Exercises
In the exercises you will develop your understanding of:
- filtering a categorical column
- using the string cache

### Exercise 1
Create a `DataFrame` from the Titanic dataset and cast the `Pclass` column to categorical.

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
    <blank>
    .head(3)
)

Continue by casting the `Embarked` column to categorical (change `with_column` to `with_columns`). 

Filter the `Pclass` column for third class passengers

Add a filter on the `Embarked` column for passengers who embarked in either Southampton (`S`) or Queenstown (`Q`)

### Exercise 2
We want to filter the Spotify `DataFrame` to find all tracks by either Taylor Swift or Ed Sheeran.

First we create the path to the CSV

In [ ]:
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"

Enable the string cache

- Create the `DataFrame`
- Cast the `artist` column to categorical
- Filter for the tracks by the artists mentioned above

In [ ]:
(
    pl.read_csv(spotify_csv,try_parse_dates=True)
    <blank>
)

Then disable the string cache

## Solutions

### Solution to Exercise 1

Cast the `Pclass` column to categorical

In [ ]:
csv_file = "../data/titanic.csv"
(
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("Pclass").cast(pl.Utf8).cast(pl.Categorical)
    )
    .head()
)


Cast the `Embarked` column to categorical

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("Pclass").cast(pl.Utf8).cast(pl.Categorical),
        pl.col("Embarked").cast(pl.Categorical)
    )
    .head(3)
)


Filter the `Pclass` column for third class passengers

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("Pclass").cast(pl.Utf8).cast(pl.Categorical),
        pl.col("Embarked").cast(pl.Categorical)
    )
    .filter(pl.col("Pclass")=="3")
    .head(3)
)


In addition, filter the `Embarked` column for passengers who embarked in Southampton (`S`) or Queenstown (`Q`)

In [ ]:
with pl.StringCache():
    df = (
        pl.read_csv(csv_file)
        .with_columns(
            pl.col("Pclass").cast(pl.Utf8).cast(pl.Categorical),
            pl.col("Embarked").cast(pl.Categorical)
        )
        .filter(pl.col("Pclass")=="3")
        .filter(pl.col("Embarked").is_in(["S","Q"]))   
    )
df.head(3)

### Solution to Exercise 2
We want to filter the Spotify `DataFrame` to find all tracks by either Taylor Swift or Ed Sheeran.

Enable the string cache (**not** using `with pl.StringCache`)

In [ ]:
pl.enable_string_cache()

In [ ]:
pl.Config.set_fmt_str_lengths(50)
spotify_csv = "../data/spotify-charts-2017-2021-global-top200.csv.gz"

- Create the `DataFrame`
- Cast the `artist` column to categorical
- Filter for the tracks by the artists mentioned above

In [ ]:
(
    pl.read_csv(spotify_csv,try_parse_dates=True)
    .with_columns(
        pl.col("artist").cast(pl.Categorical)
    )
    .filter(
        pl.col("artist").is_in(["Taylor Swift","Ed Sheeran"])
    )
    .head()
)

Then disable the string cache

In [ ]:
pl.disable_string_cache()